<a href="https://www.kaggle.com/code/ashwin8699/mobile-price-classification-using-random-forest?scriptVersionId=128125153" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mobile-price-classification/train.csv
/kaggle/input/mobile-price-classification/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')
test = pd.read_csv('/kaggle/input/mobile-price-classification/test.csv')


In [3]:
train.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


Feature Engineering 

In [4]:
train['power per wt'] = train['battery_power']/train['mobile_wt']
train['wt per vol'] = train.apply(lambda x: x['mobile_wt']/(x['sc_h']*x['sc_w']*x['m_dep']) if x['sc_w']!=0 else 0 , axis = 1)
train['power per core'] = train['battery_power']/train['n_cores']
train['pixels'] = train['px_height']*train['px_width']
train['area'] = train['sc_h']*train['sc_w']
train['px_density'] = train.apply(lambda x:x['pixels']/x['area'] if x['area']!=0 else 0 , axis = 1)
train['sc_size'] = np.sqrt(train['sc_h']**2 + train['sc_w']**2)
train['time'] = train['battery_power']/train['talk_time']
train['power per clock'] = train['battery_power']/train['clock_speed']

In [5]:
train.isna().sum()

battery_power      0
blue               0
clock_speed        0
dual_sim           0
fc                 0
four_g             0
int_memory         0
m_dep              0
mobile_wt          0
n_cores            0
pc                 0
px_height          0
px_width           0
ram                0
sc_h               0
sc_w               0
talk_time          0
three_g            0
touch_screen       0
wifi               0
price_range        0
power per wt       0
wt per vol         0
power per core     0
pixels             0
area               0
px_density         0
sc_size            0
time               0
power per clock    0
dtype: int64

In [6]:
cols = ['battery_power','clock_speed','mobile_wt','n_cores','px_height','px_width','ram','sc_w','talk_time','power per wt','wt per vol', 'power per core','pixels','area','px_density','sc_size','time','power per clock',"price_range"] 

I am using a Random Forest Model with hyperparameter tuning 

In [7]:
train2 = train[cols]
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
train , test = train_test_split(train2, test_size = 0.25)

x_train = train.drop('price_range', axis=1)
y_train = train['price_range']

x_test = test.drop('price_range', axis = 1)
y_test = test['price_range']
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
param_grid = {'n_estimators':range(10,110,10) , 'criterion':['gini', 'entropy'],
             'max_depth' : [1,2,3,4,5,6,7,8,9,10],'max_features':['auto', 'sqrt', 'log2']}

clf = RandomizedSearchCV(RandomForestClassifier(),param_grid,cv = 10,n_iter = 30)
clf = clf.fit(x_train, y_train)

In [8]:
from sklearn.metrics import classification_report,accuracy_score
accuracy_score(y_test , clf.predict(x_test))

0.896

In [9]:
from sklearn.metrics import classification_report,accuracy_score
accuracy_score(y_train , clf.predict(x_train))

1.0

From the training and testing accuracies we can say that the model is kind of overfit but does the job 